In [1]:
import morphs
%matplotlib inline
import seaborn as sns
import pandas as pd
import numpy as np
import itertools

/usr/local/anaconda/envs/morphs/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
accuracies, cluster_accuracies = morphs.load.cluster_accuracies()
good_recs = morphs.data.accuracies.good_recs()

In [3]:
# try to find nice block used below, else use last one found
for block_path in morphs.paths.blocks():
    if 'Pen01_Lft_AP2500_ML750__Site04_Z2300__B1101_cat_P01_S04_3' in block_path:
        break
subj = morphs.data.parse.bird_id(block_path)
good_clusters = morphs.data.accuracies.good_clusters(block_path)

In [4]:
spikes = morphs.load.ephys_data(block_path, good_clusters=good_clusters)

In [5]:
X, labels = morphs.spikes.create_neural_rep(spikes, max_id_len=5)
X_red = morphs.data.neurometric.logistic_dim_reduction(X, labels)
# psychometric_params = morphs.load.psychometric_params()

In [6]:
spects = morphs.load.morph_spectrograms()

In [7]:
spect_reps = {'%s%s%03d' % (l, g, i): spects[l][g][i][:,:,0] for l in spects for g in spects[l] for i in spects[l][g]}

In [8]:
def parse_morph_dim(df, morph_dim='morph_dim', lesser_dim='lesser_dim', greater_dim='greater_dim'):
    df[lesser_dim] = df[morph_dim].str[0]
    df[greater_dim] = df[morph_dim].str[1]

In [9]:
label_df = pd.DataFrame(data={'stim_id':labels})
morphs.data.parse.stim_id(label_df)

In [10]:
df_list = []
for morph_dim, group in label_df.groupby('morph_dim'):
    df_list.append(pd.DataFrame.from_records([(morph_dim, i1, i2) for i1, i2 in itertools.combinations(group.index.values, 2)], 
                                             columns=('morph_dim', 'lesser_index', 'greater_index')))
pair_df = pd.concat(df_list, ignore_index=True)
parse_morph_dim(pair_df)

In [11]:
for index in ['greater', 'lesser']:
    pair_df[index + '_morph_pos'] = label_df.loc[pair_df[index + '_index'].values, 'morph_pos'].values

In [12]:
pair_df['morph_dist'] = pair_df['greater_morph_pos'] - pair_df['lesser_morph_pos']

In [13]:
pair_df['spect_euclidean_dist'] = (pair_df['morph_dim'].str.cat(pair_df['greater_morph_pos'].map(lambda x: '%03d' % (x))).map(spect_reps) - 
                                           pair_df['morph_dim'].str.cat(pair_df['lesser_morph_pos'].map(lambda x: '%03d' % (x))).map(spect_reps)).apply(np.linalg.norm)

In [14]:
pair_df['neural_euclidian_dist'] = np.linalg.norm(X[pair_df['greater_index'].values, :] - X[pair_df['lesser_index'].values, :], axis=1)

In [15]:
pair_df['red_neural_euclidian_dist'] = np.linalg.norm(X_red[pair_df['greater_index'].values, :] - X_red[pair_df['lesser_index'].values, :], axis=1)

In [16]:
for col in ['lesser_index', 'greater_index', 'lesser_dim', 'greater_dim']:
    del pair_df[col]

In [17]:
for col in ['greater_morph_pos', 'lesser_morph_pos', 'morph_dist']:
    pair_df[col] = pair_df[col].astype('uint8')

In [18]:
pair_df['block_path'] = block_path

In [19]:
pair_df.memory_usage(deep=True)/1024/1024

Index                         0.000069
morph_dim                    30.972290
greater_morph_pos             0.455475
lesser_morph_pos              0.455475
morph_dist                    0.455475
spect_euclidean_dist          3.643799
neural_euclidian_dist         3.643799
red_neural_euclidian_dist     3.643799
block_path                   63.766479
dtype: float64

In [20]:
for col in ['morph_dim', 'block_path']:
    pair_df[col] = pair_df[col].astype('category')
pair_df['block_path'].cat.set_categories(morphs.data.accuracies.good_recs());

In [21]:
pair_df.dtypes

morph_dim                    category
greater_morph_pos               uint8
lesser_morph_pos                uint8
morph_dist                      uint8
spect_euclidean_dist          float64
neural_euclidian_dist         float64
red_neural_euclidian_dist     float64
block_path                   category
dtype: object

In [22]:
pair_df.memory_usage(deep=True)/1024/1024

Index                        0.000069
morph_dim                    0.457642
greater_morph_pos            0.455475
lesser_morph_pos             0.455475
morph_dist                   0.455475
spect_euclidean_dist         3.643799
neural_euclidian_dist        3.643799
red_neural_euclidian_dist    3.643799
block_path                   0.455685
dtype: float64